# Test imputation on untyped SNPs chrom 18

In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using StatsBase
using CodecZlib
using ProgressMeter
using JLD2, FileIO, JLSO
using BenchmarkTools
using GroupSlices
using TimerOutputs
using LinearAlgebra

### Memory requirement

**Prephasing step:** 
+ Target data requies $people * snps * 4$ bytes of RAM
+ Reference haplotype data requires $haplotypes * snps$ bits of RAM
+ Redundant haplotype set for imputation target requires roughly
$people * windows * 1000$ (max haplotypes per win) $* 16 bytes$ of RAM

## Generate subset of markers for prephasing

In [3]:
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function filter_and_mask()
    for chr in [18]
        # filter chromosome data for unique snps
        data = "../beagle_raw/chr$chr.1kg.phase3.v5a.vcf.gz"
        full_record_index = .!find_duplicate_marker(data)
        @time VCFTools.filter(data, full_record_index, 1:nsamples(data), 
            des = "chr$chr.uniqueSNPs.vcf.gz")

        # summarize data
        total_snps, samples, _, _, _, maf_by_record, _ = gtstats("chr$chr.uniqueSNPs.vcf.gz")

        # generate target panel with all snps
        n = 100
        sample_idx = falses(samples)
        sample_idx[1:n] .= true
        shuffle!(sample_idx)
        @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", 1:total_snps, 
            sample_idx, des = "target.chr$chr.full.vcf.gz", allow_multiallelic=false)

        # also generate reference panel without target samples
        @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", 1:total_snps, 
            .!sample_idx, des = "ref.chr$chr.excludeTarget.vcf.gz", allow_multiallelic=false)
        
        for maf in [0.0005, 0.001, 0.01, 0.1, 0.45]
            
            # generate target file with 100 samples and typed snps with certain maf
            my_maf = findall(x -> x > maf, maf_by_record)  
            p = length(my_maf)
            record_idx = falses(total_snps)
            record_idx[my_maf] .= true
            @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", record_idx, sample_idx, 
                des = "target.chr$chr.typedOnly.maf$maf.vcf.gz", allow_multiallelic=false)

            # unphase and mask 1% entries in target file
            masks = falses(p, n)
            missingprop = 0.001
            for j in 1:n, i in 1:p
                rand() < missingprop && (masks[i, j] = true)
            end
            @time mask_gt("target.chr$chr.typedOnly.maf$maf.vcf.gz", masks, 
                des="target.chr$chr.typedOnly.maf$maf.masked.vcf.gz", unphase=true)
        end
    end 
end
Random.seed!(2020)
@time filter_and_mask()

752.076342 seconds (6.51 G allocations: 615.353 GiB, 7.01% gc time)


Progress: 100%|█████████████████████████████████████████| Time: 0:08:17


549.100648 seconds (6.86 G allocations: 650.723 GiB, 10.18% gc time)
1139.017251 seconds (16.68 G allocations: 1.249 TiB, 10.94% gc time)
599.893049 seconds (6.86 G allocations: 650.911 GiB, 10.61% gc time)
 36.158916 seconds (267.39 M allocations: 27.452 GiB, 6.96% gc time)
608.676819 seconds (6.83 G allocations: 648.780 GiB, 10.30% gc time)
 35.610763 seconds (249.89 M allocations: 25.655 GiB, 6.86% gc time)
560.687780 seconds (6.61 G allocations: 626.620 GiB, 10.27% gc time)
 16.352906 seconds (119.38 M allocations: 12.252 GiB, 6.40% gc time)
520.047088 seconds (6.51 G allocations: 615.447 GiB, 10.21% gc time)
  8.014085 seconds (53.29 M allocations: 5.482 GiB, 5.36% gc time)
516.890704 seconds (6.43 G allocations: 607.361 GiB, 10.60% gc time)
  0.831218 seconds (5.35 M allocations: 563.018 MiB, 5.66% gc time)
6410.931663 seconds (76.92 G allocations: 6.823 TiB, 10.06% gc time)


# Try jlso compression

In [8]:
widths = [64, 128, 256, 512, 1024, 2048]
mafs = [0.1]

reffile = "ref.chr18.excludeTarget.vcf.gz"
H, H_sampleID, H_chr, H_pos, H_ids, H_ref, H_alt = convert_ht(Bool, reffile, trans=true, save_snp_info=true, msg="importing reference data...")

for maf in mafs
    tgtfile = "target.chr18.typedOnly.maf$maf.masked.vcf.gz"
    X, X_sampleID, X_chr, X_pos, X_ids, X_ref, X_alt = VCFTools.convert_gt(UInt8, tgtfile, trans=true, save_snp_info=true, msg = "Importing genotype file...")
    for width in widths
        outfile = "ref.chr18.$width.maf$maf.excludeTarget.jlso"
        @time compress_haplotypes(H, X, outfile, X_pos, H_sampleID, H_chr, H_pos, H_ids, H_ref, H_alt, width)
    end
end

importing reference data...100%|████████████████████████| Time: 0:07:21


320.246937 seconds (2.59 M allocations: 174.146 GiB, 11.52% gc time)
181.169073 seconds (2.20 M allocations: 89.185 GiB, 10.73% gc time)
144.922225 seconds (1.99 M allocations: 46.982 GiB, 11.61% gc time)
127.773475 seconds (1.86 M allocations: 25.765 GiB, 10.90% gc time)
119.267469 seconds (1.79 M allocations: 15.044 GiB, 10.64% gc time)
119.533021 seconds (1.76 M allocations: 9.956 GiB, 8.93% gc time)


In [15]:
;ls -al ref.chr18.excludeTarget.vcf.gz

-rw-r--r--@ 1 biona001  staff  247072174 May 19 23:43 ref.chr18.excludeTarget.vcf.gz


In [9]:
;ls -al ref.chr18.2048.maf0.1.excludeTarget.jlso

-rw-r--r--  1 biona001  staff  181178930 Jun 22 22:13 ref.chr18.2048.maf0.1.excludeTarget.jlso


In [10]:
;ls -al ref.chr18.1024.maf0.1.excludeTarget.jlso

-rw-r--r--  1 biona001  staff  147794707 Jun 22 22:11 ref.chr18.1024.maf0.1.excludeTarget.jlso


In [11]:
;ls -al ref.chr18.512.maf0.1.excludeTarget.jlso

-rw-r--r--  1 biona001  staff  124897790 Jun 22 22:09 ref.chr18.512.maf0.1.excludeTarget.jlso


In [12]:
;ls -al ref.chr18.256.maf0.1.excludeTarget.jlso

-rw-r--r--  1 biona001  staff  108975253 Jun 22 22:07 ref.chr18.256.maf0.1.excludeTarget.jlso


In [13]:
;ls -al ref.chr18.128.maf0.1.excludeTarget.jlso

-rw-r--r--  1 biona001  staff  104953004 Jun 22 22:04 ref.chr18.128.maf0.1.excludeTarget.jlso


In [14]:
;ls -al ref.chr18.64.maf0.1.excludeTarget.jlso

-rw-r--r--  1 biona001  staff  116884792 Jun 22 22:01 ref.chr18.64.maf0.1.excludeTarget.jlso


# MendelImpute error rate and timing

In [2]:
Threads.nthreads()

8

In [5]:
# 8 threads, keep best happair
Random.seed!(2020)
chr = 18
maf = 0.1
width = 512
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.chr18.$width.maf$maf.excludeTarget.jlso"
outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
@time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width)

X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
n, p = size(X_complete)
X_mendel = convert_gt(UInt8, outfile)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:25
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 331, averaging ~ 4096 unique haplotypes per window.

Timings: 
    Data import                     = 9.74024 seconds
    Computing haplotype pair        = 85.2919 seconds
        BLAS3 mul! to get M and N      = 8.61617 seconds (on thread 1)
        haplopair search               = 64.4019 seconds (on thread 1)
        supplying constant terms       = 0.00321546 seconds (on thread 1)
        finding redundant happairs     = 0.0970909 seconds (on thread 1)
    Phasing by dynamic programming  = 0.649192 seconds
    Imputation                      = 8.47943 seconds

104.170289 seconds (52.61 M allocations: 29.689 GiB, 5.50% gc time)
error overall = 0.006451474427693109 



In [3]:
# 8 threads, keep best happair
Random.seed!(2020)
chr = 18
maf = 0.1
width = 512
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.chr18.$width.maf$maf.excludeTarget.jlso"
outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
@time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width, screen=false)

X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
n, p = size(X_complete)
X_mendel = convert_gt(UInt8, outfile)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:16
Writing to file...100%|█████████████████████████████████| Time: 0:00:05


Total windows = 331, averaging ~ 4096 unique haplotypes per window.

Timings: 
    Data import                     = 10.966 seconds
    Computing haplotype pair        = 76.6596 seconds
        BLAS3 mul! to get M and N      = 8.16239 seconds (on thread 1)
        haplopair search               = 56.776 seconds (on thread 1)
        min least sq on observed data  = 0.00303375 seconds (on thread 1)
        finding redundant happairs     = 0.124476 seconds (on thread 1)
    Phasing by dynamic programming  = 1.07163 seconds
    Imputation                      = 7.50414 seconds

117.950050 seconds (132.68 M allocations: 33.714 GiB, 5.62% gc time)
error overall = 0.006451474427693109 



In [4]:
# 8 threads, keep best happair
Random.seed!(2020)
chr = 18
maf = 0.1
width = 512
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.chr18.$width.maf$maf.excludeTarget.jlso"
outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
@time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width, screen=false)

X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
n, p = size(X_complete)
X_mendel = convert_gt(UInt8, outfile)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:45
Writing to file...100%|█████████████████████████████████| Time: 0:00:05


Total windows = 331, averaging ~ 4096 unique haplotypes per window.

Timings: 
    Data import                     = 9.2194 seconds
    Computing haplotype pair        = 105.388 seconds
        BLAS3 mul! to get M and N      = 9.60363 seconds (on thread 1)
        haplopair search               = 46.1772 seconds (on thread 1)
        min least sq on observed data  = 0.00231333 seconds (on thread 1)
        finding redundant happairs     = 0.463115 seconds (on thread 1)
    Phasing by dynamic programming  = 0.0962801 seconds
    Imputation                      = 7.27937 seconds

121.982448 seconds (51.69 M allocations: 29.646 GiB, 15.75% gc time)
error overall = 0.006451474427693109 



In [4]:
# 8 threads, keep 100 happairs and screen using observed entries only
Random.seed!(2020)
chr = 18
maf = 0.1
width = 512
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.chr18.$width.maf$maf.excludeTarget.jlso"
outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
@time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width, screen=true)

X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
n, p = size(X_complete)
X_mendel = convert_gt(UInt8, outfile)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Computing optimal haplotype pairs...100%|███████████████| Time: 0:02:26
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 331, averaging ~ 4096 unique haplotypes per window.

Timings: 
    Data import                     = 9.58176 seconds
    Computing haplotype pair        = 146.002 seconds
        BLAS3 mul! to get M and N      = 10.7147 seconds (on thread 1)
        haplopair search               = 68.3116 seconds (on thread 1)
        min least sq on observed data  = 12.3559 seconds (on thread 1)
        finding redundant happairs     = 0.0778098 seconds (on thread 1)
    Phasing by dynamic programming  = 0.140034 seconds
    Imputation                      = 8.56125 seconds

164.284161 seconds (51.79 M allocations: 29.704 GiB, 11.40% gc time)
error overall = 0.006390132793495675 



In [3]:
# 8 threads, keep best happair
Random.seed!(2020)
function run()
    chr = 18
    maf = 0.1
    X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
    n, p = size(X_complete)
    for width in [64, 128, 256, 512, 1024, 2048]
        tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
        reffile = "ref.chr18.$width.maf$maf.excludeTarget.jlso"
        outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
        @time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width)
        
        X_mendel = convert_gt(UInt8, outfile)
        println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")
    end
end
run()

Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:37
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:37
Writing to file...100%|█████████████████████████████████| Time: 0:00:05


Total windows = 2654, averaging ~ 528 unique haplotypes per window.

Timings: 
    Data import                     = 10.5957 seconds
    Computing haplotype pair        = 37.8185 seconds
        BLAS3 mul! to get M and N      = 1.29507 seconds (on thread 1)
        haplopair search               = 22.2011 seconds (on thread 1)
        supplying constant terms       = 0.00197563 seconds (on thread 1)
        finding redundant happairs     = 2.26997 seconds (on thread 1)
    Phasing by dynamic programming  = 37.4118 seconds
    Imputation                      = 8.13028 seconds

 94.014567 seconds (66.83 M allocations: 23.407 GiB, 9.55% gc time)
error overall = 0.006065514741931171 



Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:22
Merging breakpoints...100%|█████████████████████████████| Time: 0:00:10
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total windows = 1327, averaging ~ 1268 unique haplotypes per window.

Timings: 
    Data import                     = 10.8014 seconds
    Computing haplotype pair        = 82.5595 seconds
        BLAS3 mul! to get M and N      = 3.96076 seconds (on thread 1)
        haplopair search               = 49.5698 seconds (on thread 1)
        supplying constant terms       = 0.00259007 seconds (on thread 1)
        finding redundant happairs     = 0.334425 seconds (on thread 1)
    Phasing by dynamic programming  = 10.2457 seconds
    Imputation                      = 8.63318 seconds

112.239048 seconds (53.49 M allocations: 22.345 GiB, 12.42% gc time)
error overall = 0.005058866856986026 



Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:51
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


...

Total windows = 331, averaging ~ 4096 unique haplotypes per window.

Timings: 
    Data import                     = 10.5841 seconds
    Computing haplotype pair        = 104.217 seconds
        BLAS3 mul! to get M and N      = 10.1559 seconds (on thread 1)
        haplopair search               = 77.6486 seconds (on thread 1)
        supplying constant terms       = 0.00415559 seconds (on thread 1)
        finding redundant happairs     = 0.131554 seconds (on thread 1)
    Phasing by dynamic programming  = 0.129944 seconds
    Imputation                      = 7.90087 seconds

122.831245 seconds (51.69 M allocations: 29.646 GiB, 6.03% gc time)
error overall = 0.006451474427693109 



Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:01
Writing to file...100%|█████████████████████████████████| Time: 0:00:07


Total windows = 165, averaging ~ 4743 unique haplotypes per window.

Timings: 
    Data import                     = 9.84639 seconds
    Computing haplotype pair        = 61.125 seconds
        BLAS3 mul! to get M and N      = 13.4716 seconds (on thread 1)
        haplopair search               = 38.8376 seconds (on thread 1)
        supplying constant terms       = 0.00378809 seconds (on thread 1)
        finding redundant happairs     = 0.0269305 seconds (on thread 1)
    Phasing by dynamic programming  = 0.116395 seconds
    Imputation                      = 9.17756 seconds

 80.265040 seconds (51.55 M allocations: 22.118 GiB, 6.68% gc time)
error overall = 0.009391568398854331 



Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:35
Writing to file...100%|█████████████████████████████████| Time: 0:00:09


Total windows = 82, averaging ~ 4807 unique haplotypes per window.

Timings: 
    Data import                     = 9.9829 seconds
    Computing haplotype pair        = 35.9086 seconds
        BLAS3 mul! to get M and N      = 13.7177 seconds (on thread 1)
        haplopair search               = 18.4346 seconds (on thread 1)
        supplying constant terms       = 0.00371831 seconds (on thread 1)
        finding redundant happairs     = 0.011243 seconds (on thread 1)
    Phasing by dynamic programming  = 0.150188 seconds
    Imputation                      = 11.8006 seconds

 57.842213 seconds (51.50 M allocations: 15.128 GiB, 5.62% gc time)
error overall = 0.01400232464854645 



# Haplotype Thinning

In [2]:
Threads.nthreads()

8

In [3]:
# 8 threads, hamming, search top 100 unique haplotypes per window
Random.seed!(2020)
chr = 18
maf = 0.1
width = 2000
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.$width.chr18.excludeTarget.jlso"
outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
@time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width)

X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
n, p = size(X_complete)
X_mendel = convert_gt(UInt8, outfile)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:20


Each window have ~ 4206 unique haplotypes on average


Writing to file...100%|█████████████████████████████████| Time: 0:00:05


Data import time                    = 10.2285 seconds
Computing haplotype pair time       = 20.9888 seconds
Phasing by dynamic programming time = 0.962151 seconds
Imputing time                       = 7.64746 seconds
 61.483484 seconds (316.41 M allocations: 22.896 GiB, 7.52% gc time)
error overall = 0.015155699845883544 



In [6]:
# 8 threads, hamming, search top 1000 unique haplotypes per window
Random.seed!(2020)
chr = 18
maf = 0.1
width = 2000
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.$width.chr18.excludeTarget.jlso"
outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
@time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width)

X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
n, p = size(X_complete)
X_mendel = convert_gt(UInt8, outfile)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:56


Each window have ~ 4206 unique haplotypes on average


Writing to file...100%|█████████████████████████████████| Time: 0:00:05


Data import time                    = 8.30653 seconds
Computing haplotype pair time       = 116.482 seconds
Phasing by dynamic programming time = 0.150957 seconds
Imputing time                       = 7.68015 seconds
132.606046 seconds (231.05 M allocations: 20.794 GiB, 4.15% gc time)
error overall = 0.009443362788264631 



In [4]:
# 8 threads, euclidean, search top 1000 unique haplotypes per window
Random.seed!(2020)
chr = 18
maf = 0.1
width = 2000
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.$width.chr18.excludeTarget.jlso"
outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
@time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width)

X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
n, p = size(X_complete)
X_mendel = convert_gt(UInt8, outfile)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:56
Writing to file...100%|█████████████████████████████████| Time: 0:00:06


Total window = 426, each with ~ 4206 unique haplotypes on avg
Data import time                    = 9.03808 seconds
Computing haplotype pair time       = 116.16 seconds
    BLAS3 mul! to get M and N           = 102.169 seconds (on thread 1)
    haplopair search                    = 7.02312 seconds (on thread 1)
    supplying constant terms            = 0.00553182 seconds (on thread 1)
    finding redundant happairs          = 0.0778568 seconds (on thread 1)
Phasing by dynamic programming time = 0.247162 seconds
Imputing time                       = 8.84356 seconds
134.422072 seconds (54.62 M allocations: 12.249 GiB, 3.16% gc time)
error overall = 0.007742639590336406 



In [3]:
# 8 threads, euclidean, search top 100 unique haplotypes per window
Random.seed!(2020)
chr = 18
maf = 0.1
width = 2000
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.$width.chr18.excludeTarget.jlso"
outfile = "mendel.imputed.dp$width.maf$maf.vcf.gz"
@time ph, hs = phase(tgtfile, reffile, outfile=outfile, impute=true, width=width)

X_complete = convert_gt(UInt8, "target.chr18.full.vcf.gz")
n, p = size(X_complete)
X_mendel = convert_gt(UInt8, outfile)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:08


Each window have ~ 4206 unique haplotypes on average


Writing to file...100%|█████████████████████████████████| Time: 0:00:05


Data import time                    = 8.4482 seconds
Computing haplotype pair time       = 8.34869 seconds
Phasing by dynamic programming time = 0.16332 seconds
Imputing time                       = 7.12257 seconds
 24.160206 seconds (54.39 M allocations: 10.095 GiB, 14.30% gc time)
error overall = 0.012632811088878516 



# Beagle 5.0

In [3]:
# beagle 5
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function beagle()
    chr = 18
    for maf in [0.0005, 0.001, 0.01, 0.1, 0.45]
        tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
        reffile = "ref.chr$chr.excludeTarget.vcf.gz"
        outfile = "beagle.imputed.maf$maf"
        Base.run(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=$tgtfile ref=$reffile out=$outfile nthreads=4`)
    end
end
beagle()

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 10:09 AM PDT on 08 May 2020

Command line: java -Xmx13653m -jar beagle.28Sep18.793.jar
  gt=target.chr18.typedOnly.maf0.0005.masked.vcf.gz
  ref=ref.chr18.excludeTarget.vcf.gz
  out=beagle.imputed.maf0.0005
  nthreads=4

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,404
Study samples:             100

Window 1 (18:10644-40010629)
Reference markers:     414,911
Study markers:         414,911

Burnin  iteration 1:           48 seconds
Burnin  iteration 2:           1 minute 28 seconds
Burnin  iteration 3:           1 minute 42 seconds
Burnin  iteration 4:           1 minute 46 seconds
Burnin  iteration 5:           1 minute 53 seconds
Burnin  iteration 6:           2 minutes 5 seconds

Phasing iteration 1:           1 minute 59 seconds
Phasing iteration 2:           2 minutes 1 second
Phasing iterati

DimensionMismatch: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 678557 and 863592")

In [5]:
for maf in [0.0005]
    # beagle error rate    
    chr = 18
    X_complete = convert_gt(UInt8, "target.chr$chr.full.vcf.gz")
    X_beagle = convert_gt(UInt8, "beagle.imputed.maf$maf.vcf.gz")
    n, p = size(X_complete)
    println("error overall = $(sum(X_beagle .!= X_complete) / n / p) \n")
end

error overall = 5.453964372064586e-6 



In [1]:
# beagle 5
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function beagle()
    chr = 18
    for maf in [0.001, 0.01, 0.1, 0.45]
        tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
        reffile = "ref.chr$chr.excludeTarget.vcf.gz"
        outfile = "beagle.imputed.maf$maf"
        Base.run(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=$tgtfile ref=$reffile out=$outfile nthreads=4`)
    end
end
beagle()

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 02:52 PM PDT on 12 May 2020

Command line: java -Xmx13653m -jar beagle.28Sep18.793.jar
  gt=target.chr18.typedOnly.maf0.001.masked.vcf.gz
  ref=ref.chr18.excludeTarget.vcf.gz
  out=beagle.imputed.maf0.001
  nthreads=4

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,404
Study samples:             100

Window 1 (18:10644-40010629)
Reference markers:     414,911
Study markers:         388,114

Burnin  iteration 1:           38 seconds
Burnin  iteration 2:           1 minute 26 seconds
Burnin  iteration 3:           1 minute 28 seconds
Burnin  iteration 4:           1 minute 35 seconds
Burnin  iteration 5:           1 minute 50 seconds
Burnin  iteration 6:           1 minute 47 seconds

Phasing iteration 1:           1 minute 43 seconds
Phasing iteration 2:           1 minute 47 seconds
Phasing iteratio

UndefVarError: UndefVarError: convert_gt not defined

In [3]:
# beagle error rate    
chr = 18
X_complete = convert_gt(UInt8, "target.chr$chr.full.vcf.gz")
X_beagle = convert_gt(UInt8, "beagle.imputed.maf0.001.vcf.gz")
n, p = size(X_complete)
println("error overall = $(sum(X_beagle .!= X_complete) / n / p) \n")

ErrorException: GeneticVariation.VCF.Reader file format error on line 11 ~>"�;IMP\t"

In [7]:
# beagle 5
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function beagle()
    chr = 18
    for maf in [0.01]
        tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
        reffile = "ref.chr$chr.excludeTarget.vcf.gz"
        outfile = "beagle.imputed.maf$maf"
        Base.run(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=$tgtfile ref=$reffile out=$outfile nthreads=4`)
    
        # beagle error rate    
        chr = 18
        X_complete = convert_gt(UInt8, "target.chr$chr.full.vcf.gz")
        X_beagle = convert_gt(UInt8, "beagle.imputed.maf$maf.vcf.gz")
        n, p = size(X_complete)
        println("error overall = $(sum(X_beagle .!= X_complete) / n / p) \n")    
    end
end
beagle()

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 04:03 PM PDT on 12 May 2020

Command line: java -Xmx13653m -jar beagle.28Sep18.793.jar
  gt=target.chr18.typedOnly.maf0.01.masked.vcf.gz
  ref=ref.chr18.excludeTarget.vcf.gz
  out=beagle.imputed.maf0.01
  nthreads=4

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,404
Study samples:             100

Window 1 (18:10644-40010629)
Reference markers:     414,911
Study markers:         183,460

Burnin  iteration 1:           22 seconds
Burnin  iteration 2:           31 seconds
Burnin  iteration 3:           34 seconds
Burnin  iteration 4:           33 seconds
Burnin  iteration 5:           38 seconds
Burnin  iteration 6:           38 seconds

Phasing iteration 1:           37 seconds
Phasing iteration 2:           35 seconds
Phasing iteration 3:           42 seconds
Phasing iteration 4:           38 secon

ErrorException: GeneticVariation.VCF.Reader file format error on line 11 ~>"�;IMP\t"

In [8]:
chr = 18
maf = 0.01
X_complete = convert_gt(UInt8, "target.chr$chr.full.vcf.gz")
X_beagle = convert_gt(UInt8, "beagle.imputed.maf$maf.vcf.gz")
n, p = size(X_complete)
println("error overall = $(sum(X_beagle .!= X_complete) / n / p) \n")    

error overall = 0.0011677968299845297 



In [9]:
# beagle 5
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function beagle()
    chr = 18
    for maf in [0.1, 0.45]
        tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
        reffile = "ref.chr$chr.excludeTarget.vcf.gz"
        outfile = "beagle.imputed.maf$maf"
        Base.run(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=$tgtfile ref=$reffile out=$outfile nthreads=4`)
    
        # beagle error rate
        chr = 18
        X_complete = convert_gt(UInt8, "target.chr$chr.full.vcf.gz")
        X_beagle = convert_gt(UInt8, "beagle.imputed.maf$maf.vcf.gz")
        n, p = size(X_complete)
        println("error overall = $(sum(X_beagle .!= X_complete) / n / p) \n")
    end
end
beagle()

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 04:47 PM PDT on 12 May 2020

Command line: java -Xmx13653m -jar beagle.28Sep18.793.jar
  gt=target.chr18.typedOnly.maf0.1.masked.vcf.gz
  ref=ref.chr18.excludeTarget.vcf.gz
  out=beagle.imputed.maf0.1
  nthreads=4

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,404
Study samples:             100

Window 1 (18:10644-40010629)
Reference markers:     414,911
Study markers:          82,786

Burnin  iteration 1:           18 seconds
Burnin  iteration 2:           19 seconds
Burnin  iteration 3:           19 seconds
Burnin  iteration 4:           19 seconds
Burnin  iteration 5:           22 seconds
Burnin  iteration 6:           19 seconds

Phasing iteration 1:           19 seconds
Phasing iteration 2:           18 seconds
Phasing iteration 3:           18 seconds
Phasing iteration 4:           17 seconds

# Eagle 2 + Minimac4

In order to use the reference panel in Eagle 2's prephase option, one must first convert it to `.bcf` format via e.g. `htslib` which is *extremely* difficult to install. Even after we went through all the hard work to obtain the final `.bcf` reference file (see commands below), eagle 2.4 STILL SAYS the file is not acceptable (not bgzipped or some processing error). Therefore, I have no choice but to prephase without the reference panel. 

In [ ]:
# run eagle 2.4: 3367.79 sec on amd-2382 machine (can only run on linux systems)
eagle --vcf=target.chr20.typedOnly.masked.vcf.gz --outPrefix=eagle.phased.chr20 --numThreads=4 --geneticMapFile=../Eagle_v2.4.1/tables/genetic_map_hg19_withX.txt.gz

In [ ]:
# convert ref file to m3vcf format (Total Run completed in 1 hours, 46 mins, 24 seconds)
/u/home/b/biona001/haplotype_comparisons/Minimac3/bin/Minimac3 --refHaps ref.chr20.excludeTarget.vcf.gz --processReference --prefix ref.chr20.excludeTarget

In [ ]:
# run minimac4 (2619 seconds)
minimac4 --refHaps ref.chr20.excludeTarget.m3vcf.gz --haps eagle.phased.vcf.gz --prefix minimac.imputed.chr20 --format GT --cpus 4

In [ ]:
# minimac4 error rate    
X_complete = convert_gt(Float32, "target.chr20.full.vcf.gz")
X_minimac = convert_gt(Float32, "minimac.imputed.chr20.dose.vcf.gz")
n, p = size(X_complete)
println("error overall = $(sum(X_minimac .!= X_complete) / n / p) \n")